In [1]:
x_min    = -2
x_max    =  1
y_min    = -1.5
y_max    =  1.5
width    =  2000
height   =  2000
max_iter =  50000

In [2]:
from numba import njit, prange
import numpy as np
from math import sin, cos, pi
from random import uniform
import time

@njit(fastmath=True)
def create_rand_point():
    ###
    # polar coordinates
    r = uniform(0, 2)
    phi = uniform(0, 2*pi)
    return (r*cos(phi), r*sin(phi))

@njit(fastmath=True)
def get_pixel_coord(z_real, z_imag):
    ###
    # compute pixel density
    x_dens = (width - 1) / abs(x_max - x_min)
    y_dens = (height - 1) / abs(y_max - y_min)
    ###
    # compute nearest pixel
    x_pixel = round(x_dens * (z_real - x_min))
    y_pixel = width - 1 - round(y_dens * (z_imag - y_min)) # image starts top left
    ###
    # does nearest pixel exceed image size?
    if x_pixel < 0 or x_pixel > width -1 or y_pixel < 0 or y_pixel > height - 1:
        return None

    return (x_pixel, y_pixel)

@njit
def get_new_path():
    ###
    # create random point
    (c_real, c_imag) = create_rand_point()
    ###
    # first step
    z_real = c_real
    z_imag = c_imag
    ###
    # establish path
    path = np.zeros((width,height), np.int32)
    ###
    # start iteration
    for i in range(max_iter):
        z_real2 = z_real * z_real
        z_imag2 = z_imag * z_imag
        ###
        # does zn surely diverge?
        if z_real2 + z_imag2 > 4.0:
            return np.zeros((width,height), np.int32)
        ###
        # get nearest pixel
        pixel = get_pixel_coord(z_real, z_imag)
        ###
        # pixel already visited?
        if pixel is not None:
            (x, y) = pixel
            path[x][y] += 1
        ###
        # prepare new step
        z_imag = 2 * z_real*z_imag + c_imag
        z_real = z_real2 - z_imag2 + c_real
    return path

@njit(parallel=True)
def sharpen_buddha(loops=1):
    ###
    # define density
    density = np.zeros((width,height), np.int64)
    ###
    # start the loop
    for i in prange(loops):
        density += get_new_path()
    return density

def sharpen_buddha_save(loops=1, steps=1):
    ###
    # establish file names
    file_name     = f"buddhabrot_{x_min}_{x_max}_{y_min}_{y_max}_{width}_{height}_{max_iter}"
    file_name_np  = file_name + '.npy'
    file_name_log = file_name + '.txt'
    ###
    # load files
    try:
        density = np.load(file_name_np)
        print_update('Old numpy file found and loaded...')
    except FileNotFoundError:
        density = np.zeros((width,height), np.int64)
        np.save(file_name_np, density)
        self.print_update('No numpy file found. New file created...')
    try:
        with open(file_name_log, 'r') as f:
            num_eval_points = eval(f.read())
            print_update('Log file found and loaded...')
            print_update(str(num_eval_points) + ' points evaluated so far...')
    except FileNotFoundError:
        with open(file_name_log, 'w') as f:
            print_update('No log file found. New file created...')
            print_update('Estimate evaluated points...')
            num_eval_points = density.sum() // max_iter
            print_update(str(num_eval_points) + ' points estimated...')
            f.write(str(num_eval_points))
    ###
    # start process    
    for i in range(steps):
        n_density = sharpen_buddha(loops=loops)
        num_eval_points += loops
        print_update(f'{loops} new points evaluated...')
        density += n_density
        np.save(file_name_np, density)
        with open(file_name_log, 'w') as f:
            f.write(str(num_eval_points))
        print_update('File saved...')
    return density

def print_update(msg):
    print(time.ctime() + ': ' + msg)

In [ ]:
%%time
sharpen_buddha_save(loops=10000, steps=60)

Thu Apr  2 11:25:19 2020: Old numpy file found and loaded...
Thu Apr  2 11:25:19 2020: Log file found and loaded...
Thu Apr  2 11:25:19 2020: 2624000 points evaluated so far...
Thu Apr  2 11:26:27 2020: 10000 new points evaluated...
Thu Apr  2 11:26:27 2020: File saved...
Thu Apr  2 11:27:33 2020: 10000 new points evaluated...
Thu Apr  2 11:27:33 2020: File saved...
Thu Apr  2 11:28:38 2020: 10000 new points evaluated...
Thu Apr  2 11:28:38 2020: File saved...
Thu Apr  2 11:29:44 2020: 10000 new points evaluated...
Thu Apr  2 11:29:44 2020: File saved...
Thu Apr  2 11:30:50 2020: 10000 new points evaluated...
Thu Apr  2 11:30:50 2020: File saved...
Thu Apr  2 11:31:56 2020: 10000 new points evaluated...
Thu Apr  2 11:31:56 2020: File saved...
Thu Apr  2 11:33:02 2020: 10000 new points evaluated...
Thu Apr  2 11:33:02 2020: File saved...
